# Libraries

Here we store the libraries needed to run patchcore correctly including wich created by us.

In [2]:
# External Modules
import torch
import torch.nn.functional as F
import torchvision
import numpy as np
import timm
import time
import utils
import tqdm
import sampler
import patchcore as PatchcoreLib
from numba import jit, cuda
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score, confusion_matrix
import os
# Created Modules
import data #load the dataset into data loader
#import network as nn #load the neural network

# GPU
Checking GPU status

In [3]:
print(torch.cuda.is_available())

True


In [4]:
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 3060 Laptop GPU


In [5]:
#Defining the device
device = torch.device("cuda:0")

In [6]:
torch.cuda.empty_cache()

# Patchcore

In [7]:
# Global variables
RESIZE = 256
IMAGESIZE = 224
PATCHSIZE = 3
PATCHSTRIDE = 1
DILATION = 1
PRETRAIN_EMBED = 1024
TARGET_EMBED = 1024
BATCH_SIZE = 1
PERCENTAGE = 0.1

## Load the datasets

- Loading the desired datasets

In [8]:
selected_data = ['wood','zipper','toothbrush','metal_nut','screw','pill','transistor','tile','leather']
datasets =  data.export_data(".\data",selected_data, RESIZE, IMAGESIZE, BATCH_SIZE)

In [9]:
#checking
datasets

{'wood': {'train': <data.Data at 0x24b4b48a1f0>,
  'test': <data.Data at 0x24b4b48a2e0>},
 'zipper': {'train': <data.Data at 0x24b4b435c70>,
  'test': <data.Data at 0x24b59f2d3d0>},
 'toothbrush': {'train': <data.Data at 0x24b59f2d760>,
  'test': <data.Data at 0x24b59f2daf0>},
 'metal_nut': {'train': <data.Data at 0x24b59f2de80>,
  'test': <data.Data at 0x24b59f7b250>},
 'screw': {'train': <data.Data at 0x24b59f7b5e0>,
  'test': <data.Data at 0x24b59f7b970>},
 'pill': {'train': <data.Data at 0x24b59f7bd00>,
  'test': <data.Data at 0x24b59fb70d0>},
 'transistor': {'train': <data.Data at 0x24b59fb7460>,
  'test': <data.Data at 0x24b59fb77f0>},
 'tile': {'train': <data.Data at 0x24b59fb7b80>,
  'test': <data.Data at 0x24b59fb7f10>},
 'leather': {'train': <data.Data at 0x24b59fec2e0>,
  'test': <data.Data at 0x24b59fec670>}}

## 2. Autoencoder - ResNet

### Params

- ResNetlike architecture with layers = {1,2,3,4}
- Pathcore uses j and j + 1 layers
- As we want mid-level features, we will use 2 and 3

In [10]:
#In general for Resnet we use those layers.
extract_layers = [2,3]

### Timm Widereset

In [11]:
name = "wide_resnet50_2"
# We need to use timm because we will use pretrained dataset
#Setting the resnet with out_indices that are our mid-level features
neural_network = timm.create_model(name, out_indices = extract_layers, features_only = True, pretrained = True)
neural_network.eval()
neural_network = neural_network.to(device)

### 3. Sampler Definition

In [12]:
# Defining the parameter of the sampler
sampling = sampler.Sampler(PERCENTAGE,device)

### Pathcore Instance

In [13]:
patchcore = PatchcoreLib.Patchcore(neural_network,sampling,
                                RESIZE,IMAGESIZE,PATCHSIZE,PATCHSTRIDE,DILATION,PRETRAIN_EMBED,TARGET_EMBED,
                                device)

### Fitting and testing Patchcore

In [14]:
start_time = time.time()
score_list = {}
for data in datasets:
    print(f"{data} begin at ")
    print("--- %s seconds ---" % (time.time() - start_time))
    patchcore.fit(datasets[data]["train"].dataloader)
    print(f"{data} fit end at ")
    print("--- %s seconds ---" % (time.time() - start_time))
    score_list[data] = patchcore.predict(datasets[data]["test"].dataloader)
    print(f"{data} end at ")
    print("--- %s seconds ---" % (time.time() - start_time))

wood begin at 
--- 0.0009989738464355469 seconds ---
wood fit end at 
--- 402.35060453414917 seconds ---


KeyboardInterrupt: 

### Analysis of patchcore scores

In [ ]:
stat = dict.fromkeys(selected_data, {})

In [ ]:
for data in datasets:
    
    print(f"--------RESULTS FOR {data}-----------")
    label =[1 if x["score"][1][0]!="good" else 0 for x in score_list[data]]
    preds = [x["score"][0].to("cpu") for x in score_list[data]]
    label = np.stack(label)
    preds = np.stack(preds)
    stat[data]["roc_auc"] = roc_auc_score(label, preds)
    print(f"roc_auc score: {stat[data]['roc_auc']}")
    print(f"We obtain the following plot: ")
    utils.ROC(label, preds, data)
    image_paths = [os.path.join(".\data",f"{data}","test",x[0],x[1]) for x in datasets[data]["test"].dataloader.dataset.data_to_iterate]
    mask_paths = [x[2] for x in datasets[data]["test"].dataloader.dataset.data_to_iterate]
    image_save_path = f".\\data\\{data}\\segmentation"
    os.makedirs(image_save_path, exist_ok = True)
    mask_list = [x["mask"][0].to("cpu") for x in score_list[data]]
    utils.plot_segmentation_images(datasets,data,image_save_path,image_paths,mask_list,preds,mask_paths)

## Training on different k

In [ ]:
d_k = {1:[],2:[],3:[],4:[],5:[]}
start_time = time.time()
score_list2 = {}
for k in range(2,6):
    
    
    patchcore = PatchcoreLib.Patchcore(neural_network,sampling,
                                    RESIZE,IMAGESIZE,PATCHSIZE,PATCHSTRIDE,DILATION,PRETRAIN_EMBED,TARGET_EMBED,
                                    device,k)
    for data in datasets:
        print(f"{data} begin at ")
        print("--- %s seconds ---" % (time.time() - start_time))
        patchcore.fit(datasets[data]["train"].dataloader)
        print(f"{data} fit end at ")
        print("--- %s seconds ---" % (time.time() - start_time))
        score_list2[data] = patchcore.predict(datasets[data]["test"].dataloader)
        print(f"{data} end at ")
        print("--- %s seconds ---" % (time.time() - start_time))
    
    for data in datasets:
        print(f"--------RESULTS FOR {data}-----------")
        label =[1 if x["score"][1][0]!="good" else 0 for x in score_list2[data]]
        preds = [x["score"][0].to("cpu") for x in score_list2[data]]
        label = np.stack(label)
        preds = np.stack(preds)
        d_k[k].append(roc_auc_score(label, preds))

In [ ]:
with open('k_2.txt','w') as f:
    f.write('dict = ' + str(d_k) + '\n')

## Training on different Percentage

In [ ]:
d_percentage = {str(0.1):[],str(0.25):[],str(0.5):[],str(1):[]}
start_time = time.time()
score_list2 = {}
for per in [1]:
    torch.cuda.empty_cache()
    sampling = sampler.Sampler(per,device)
    patchcore = PatchcoreLib.Patchcore(neural_network,sampling,
                                    RESIZE,IMAGESIZE,PATCHSIZE,PATCHSTRIDE,DILATION,PRETRAIN_EMBED,TARGET_EMBED,
                                    device)
    for data in datasets:
        torch.cuda.empty_cache()
        print(f"{data} begin at ")
        print("--- %s seconds ---" % (time.time() - start_time))
        patchcore.fit(datasets[data]["train"].dataloader)
        print(f"{data} fit end at ")
        print("--- %s seconds ---" % (time.time() - start_time))
        score_list2[data] = patchcore.predict(datasets[data]["test"].dataloader)
        print(f"{data} end at ")
        print("--- %s seconds ---" % (time.time() - start_time))
    
    for data in datasets:
        print(f"--------RESULTS FOR {data}-----------")
        label =[1 if x["score"][1][0]!="good" else 0 for x in score_list2[data]]
        preds = [x["score"][0].to("cpu") for x in score_list2[data]]
        label = np.stack(label)
        preds = np.stack(preds)
        d_percentage[str(per)].append(roc_auc_score(label, preds))

wood begin at 
--- 0.0005345344543457031 seconds ---


In [ ]:
with open('percentage_3.txt','w') as f:
    f.write('dict = ' + str(d_percentage) + '\n')

### Plotting Hyperparameters Graphs